<a href="https://colab.research.google.com/github/patrickhamzaokello/practice/blob/master/CUDA_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CUDA Neural Networks

## Demo - Binary number classification with a CUDA Neural Network

![alt text](https://i.imgur.com/FhGiI9R.png)

In [0]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true simple.cu -o simple -lcudadevrt

gcc: error: simple.cu: No such file or directory
gcc: warning: ‘-x c++’ after last input file has no effect
gcc: fatal error: no input files
compilation terminated.


In [0]:
!./simple

Prediction[0] : 0.060997 True Value[0] : 0.000000 Error[0] : 0.060997
Prediction[1] : 0.076193 True Value[1] : 0.000000 Error[1] : 0.076193
Prediction[2] : 0.927551 True Value[2] : 1.000000 Error[2] : -0.072449
Prediction[3] : 0.918263 True Value[3] : 1.000000 Error[3] : -0.081737


In [0]:
//
//  onehiddenlayerperceptron.cu
//  onehiddenlayerperceptron
//
//  Created by Sergei Bugrov on 8/21/17.
//  Copyright © 2017 Sergei Bugrov. All rights reserved.
//

#include <stdio.h>

__global__ void kMartixByMatrixElementwise(const int nThreads, const float *m1, const float *m2, float *output) {
    /*  Computes the product of two arrays (elementwise multiplication).
     Inputs:
     m1: array
     m2: array
     output: array,the results of the multiplication are to be stored here
    */
	for (int i = blockIdx.x * blockDim.x + threadIdx.x;
		 i < nThreads;
		 i += blockDim.x * gridDim.x)
	  {
		output[i] = m1[i] * m2[i];
	  }
}

__device__ float* dMartixByMatrixElementwise(const float *m1, const float *m2, float *output, const int width, const int height){

	kMartixByMatrixElementwise <<< width, height >>> ( width * height, m1, m2, output );
    cudaDeviceSynchronize();
    return output;
}

__global__ void kMartixSubstractMatrix(const int nThreads, const float *m1, const float *m2, float *output) {
    /*  Computes the (elementwise) difference between two arrays
     Inputs:
     m1: array
     m2: array
     output: array,the results of the computation are to be stored here
     */

	for (int i = blockIdx.x * blockDim.x + threadIdx.x;
		 i < nThreads;
		 i += blockDim.x * gridDim.x)
	  {
		output[i] = m1[i] - m2[i];
	  }
}

__device__ float* dMartixSubstractMatrix(const float *m1, const float *m2, float *output, const int width, const int height){

	kMartixSubstractMatrix <<< width, height >>> ( width * height, m1, m2, output );
    cudaDeviceSynchronize();
    return output;
}

__global__ void kSigmoid(const int nThreads, float const *input, float *output){
    /*  Computes the value of the sigmoid function f(x) = 1/(1 + e^-x).
     Inputs:
     input: array
     output: array, the results of the computation are to be stored here
    */

	for (int i = blockIdx.x * blockDim.x + threadIdx.x;
		 i < nThreads;
		 i += blockDim.x * gridDim.x)
	  {
		output[i] = 1.0 / (1.0 + std::exp(-input[i]));
	  }
}

__device__ void dSigmoid(float const *input, float *output, const int height, const int width){

	kSigmoid <<< height, width >>> (height * width, input, output);
	cudaDeviceSynchronize();
}

__global__ void kSigmoid_d(const int nThreads, float const *input, float *output) {
	/*  Computes the value of the sigmoid function derivative f'(x) = f(x)(1 - f(x)),
	    where f(x) is sigmoid function.
	    Inputs:
	    input: array
	    output: array, the results of the computation are to be stored here:
	    		x(1 - x) for every element of the input matrix m1.
	*/

	for (int i = blockIdx.x * blockDim.x + threadIdx.x;
		 i < nThreads;
		 i += blockDim.x * gridDim.x)
	  {
		output[i] = input[i] * (1 - input[i]);
	  }
}

__device__ float* dSigmoid_d(float const *input, float *output, const int rows, const int columns){
	kSigmoid_d <<< rows, columns >>> (rows*columns, input, output);
	cudaDeviceSynchronize();
	return output;
}

__global__ void kDot(const int nThreads, const float *m1, const float *m2, float *output, const int m1_rows , const int m1_columns, const int m2_columns ){
	/*  Computes the product of two matrices: m1 x m2.
	   	Inputs:
	    m1: array, left matrix of size m1_rows x m1_columns
	    m2: array, right matrix of size m1_columns x m2_columns (the number of rows in the right matrix
	    must be equal to the number of the columns in the left one)
	    output: array, the results of the computation are to be stored here:
	    		m1 * m2, product of two arrays m1 and m2, a matrix of size m1_rows x m2_columns
	    m1_rows: int, number of rows in the left matrix m1
	    m1_columns: int, number of columns in the left matrix m1
	    m2_columns: int, number of columns in the right matrix m2
	*/

	for (int i = blockIdx.x * blockDim.x + threadIdx.x;
		 i < nThreads;
		 i += blockDim.x * gridDim.x)
	{
	    int r = (int)i / m2_columns;
	    int c = i % m2_columns;
	    float t_output = 0.f;

	    for( int k = 0; k < m1_columns; ++k ) {
	        t_output += m1[ r * m1_columns + k ] * m2[ k * m2_columns + c ];
	    }

	    output[i] = t_output;
	}
}

__device__ float* dDot(const float *m1, const float *m2, float *output, const int m1_rows , const int m1_columns, const int m2_columns ){

	kDot <<< m1_rows, m2_columns >>> (m1_rows * m2_columns, m1, m2, output, m1_rows , m1_columns, m2_columns );
	cudaDeviceSynchronize();
	return output;
}

__global__ void kDot_m1_m2T(const int nThreads, const float *m1, const float *m2, float *output, const int m1_columns, const int m2_rows ){
	/*  Updates the output matrix with the product of two matrices: m1 and m2 transposed.
	   	Inputs:
	    m1: array, left matrix of size m1_rows x m1_columns
	    m2: array, right matrix of size m2_rows x m1_columns (m2 transposed will be of size m1_columns x m2_rows)
	    output: array, the results of the computation are to be stored here:
	    		m1 * m2, product of two arrays m1 and m2, a matrix of size m1_rows x m2_rows
	    m1_columns: int, number of columns in the left matrix m1
	    m2_rows: int, number of rows in the left matrix m2
	*/

	for (int i = blockIdx.x * blockDim.x + threadIdx.x;
		 i < nThreads;
		 i += blockDim.x * gridDim.x)
	{
		int r = (int)i / m2_rows;
		int c = i % m2_rows;
		float t_output = 0.0;
		int id_T;

		for( int k = 0; k < m1_columns; ++k ) {
			id_T = c * m1_columns + k;
			t_output += m1[ r * m1_columns + k ] * m2[ id_T ];
		}

		output[i] = t_output;
	}
}

__device__ float* dDot_m1_m2T(const float *m1, const float *m2, float *output, const int m1_rows , const int m1_columns, const int m2_rows )
{
	kDot_m1_m2T <<< m1_rows, m2_rows >>> ( m1_rows * m2_rows, m1, m2, output, m1_columns, m2_rows );
	cudaDeviceSynchronize();
	return output;
}

__global__ void kDot_m1T_m2(const int nThreads, const float *m1, const float *m2, float *output, const int m1_rows,
							const int m1_columns, const int m2_columns ){
	/*  Increments the output matrix with the product of two matrices: m1 transposed and m2.
	   	Inputs:
	    m1: array, left matrix of size m1_rows x m1_columns (m1 transposed will be of size m1_columns x m1_rows)
	    m2: array, right matrix of size m1_rows x m2_columns
	    output: array, the results of the computation are to be stored here:
	    		m1 * m2, product of two arrays m1 and m2, a matrix of size m1_columns x m2_columns
	    m1_rows: int, number of rows in the left matrix m1
	    m1_columns: int, number of columns in the left matrix m1
	    m2_rows: int, number of rows in the left matrix m2
	*/

	for (int i = blockIdx.x * blockDim.x + threadIdx.x;
		 i < nThreads;
		 i += blockDim.x * gridDim.x)
	{
	    int r = (int)i / m2_columns;
	    int c = i % m2_columns;
	    int id_T;
	    float t_output = 0.0;

	    for( int k = 0; k < m1_rows; ++k ) {
	    	id_T = k * m1_columns + r;
	        t_output += m1[ id_T ] * m2[ k * m2_columns + c ];
	    }

	    output[i] += t_output;
	}
}

__device__ void dDot_m1T_m2(const float *m1, const float *m2, float *output, const int m1_height , const int m1_width, const int m2_width )
{
	kDot_m1T_m2 <<< m1_width, m2_width >>> (m1_width * m2_width, m1, m2, output, m1_height, m1_width, m2_width );
	cudaDeviceSynchronize();
}

__device__ void kPrintMatrix (const float* M, int h, int w) {
    /*  Prints out the input array as h x w matrix.
     Inputs:
     m: vector, matrix of size n_rows x n_columns
     h: int, number of rows in the matrix M
     w: int, number of columns in the matrix M
     */
	for (int i = 0; i < h; i++){
		for (int j = 0; j < w; j++){
			printf("%f  ", M[i*w+j]);
		}
		printf("\n");
	}
	printf("\n");
}

__global__ void kFit(	const float* X, const int X_w, const int X_h,
						const float* y, const int y_w,
						float* l1, const int l1_w, float* l_1_d,
						float* pred, float* pred_d,
						float* W0,
						float* W1,
						float* buffer
						)
{
	for (unsigned i = 0; i < 50; ++i) {

        dSigmoid(dDot(X, W0, l1, X_h, X_w, l1_w), l1, X_h, l1_w);
        dSigmoid(dDot(l1, W1, pred, X_h, l1_w, y_w), pred, X_h, y_w);
        dMartixByMatrixElementwise(dMartixSubstractMatrix(y, pred, pred_d, X_h, y_w), dSigmoid_d(pred, buffer, X_h, y_w), pred_d, X_h, y_w );
        dMartixByMatrixElementwise(dDot_m1_m2T(pred_d, W1, l_1_d, X_h, y_w, l1_w), dSigmoid_d(l1, buffer, X_h, l1_w), l_1_d, X_h, l1_w);
        dDot_m1T_m2( l1, pred_d, W1, X_h, l1_w, y_w );
        dDot_m1T_m2( X, l_1_d, W0, X_h, X_w, l1_w );
    }
}

int main(void){

	const int TRAINING_SIZE = 4;
	const int TRAINING_DIM = 4;
	const int L1_SIZE = 8;

	// X, the first 4 lines from Iris dataset
	float h_X[TRAINING_SIZE*TRAINING_DIM] = {	5.1, 3.5, 1.4, 0.2,
												4.9, 3.0, 1.4, 0.2,
												6.2, 3.4, 5.4, 2.3,
												5.9, 3.0, 5.1, 1.8 };

	const signed int X_size = sizeof(h_X);

	float *d_X;
	cudaMalloc(&d_X, X_size);
	cudaMemcpy(d_X, h_X, X_size, cudaMemcpyHostToDevice);

	//WEIGHTS_0
	const long signed int W0_size = L1_SIZE*TRAINING_DIM*sizeof(float);
	float *h_W0 = (float*)malloc(W0_size);
	for (int i = 0; i < L1_SIZE*TRAINING_DIM; i++){
	    h_W0[i] = 0.1 * (2.0*rand()/RAND_MAX-1.0);
	}

	float *d_W0;
	cudaMalloc(&d_W0, W0_size);
	cudaMemcpy(d_W0, h_W0, W0_size, cudaMemcpyHostToDevice);

	//LAYER_1, LAYER_1_DELTA AND BUFFER OF LAYER 1 SIZE
	const long signed int L1_size = L1_SIZE*TRAINING_SIZE*sizeof(float);

	float* h_layer_1 = (float*)malloc(L1_size);
	float* h_layer_1_delta = (float*)malloc(L1_size);
	float* h_buffer = (float*)malloc(L1_size);

	for (int i = 0; i < L1_SIZE*TRAINING_SIZE; i++){
	    h_layer_1[i] = 0.0;
	    h_buffer[i] = 0.0;
	    h_layer_1_delta[i] = 0.0;
	}

	float *d_layer_1;
	cudaMalloc(&d_layer_1, L1_size);
	cudaMemcpy(d_layer_1, h_layer_1, L1_size, cudaMemcpyHostToDevice);

	float *d_buffer;
	cudaMalloc(&d_buffer, L1_size);
	cudaMemcpy(d_buffer, h_buffer, L1_size, cudaMemcpyHostToDevice);

	float *d_layer_1_delta;
	cudaMalloc(&d_layer_1_delta, L1_size);
	cudaMemcpy(d_layer_1_delta, h_layer_1_delta, L1_size, cudaMemcpyHostToDevice);

	//WEIGHTS_1
	const long signed int W1_size = L1_SIZE*sizeof(float);
	float *h_W1 = (float*)malloc(W1_size);
	for (int i = 0; i < L1_SIZE; i++){
	    h_W1[i] = 0.1* (2.0*rand()/RAND_MAX-1.0);
	}

	float *d_W1;
	cudaMalloc(&d_W1, W1_size);
	cudaMemcpy(d_W1, h_W1, W1_size, cudaMemcpyHostToDevice);

	//Y
	float h_y[4] = {	0,
						0,
						1,
						1 };
	const signed int y_size = sizeof(h_y);
	float *d_y;
	cudaMalloc(&d_y, y_size);
	cudaMemcpy(d_y, h_y, y_size, cudaMemcpyHostToDevice);

	//PRED AND PRED_DELTA
	float* h_pred = (float*)malloc(y_size);
	float* h_pred_delta = (float*)malloc(y_size);
	for (int i = 0; i < TRAINING_SIZE; i++){
	    h_pred[i] = 0.0;
	    h_pred_delta[i] = 0.0;
	}

	float *d_pred;
	cudaMalloc(&d_pred, y_size);
	cudaMemcpy(d_pred, h_pred, y_size, cudaMemcpyHostToDevice);

	float *d_pred_delta;
	cudaMalloc(&d_pred_delta, y_size);
	cudaMemcpy(d_pred_delta, h_pred_delta, y_size, cudaMemcpyHostToDevice);

	kFit <<< 1, 1 >>> (	d_X, TRAINING_DIM, TRAINING_SIZE,
						d_y, 1,
						d_layer_1, L1_SIZE, d_layer_1_delta,
						d_pred,
						d_pred_delta,
						d_W0,
						d_W1,
						d_buffer);

	cudaMemcpy(h_pred, d_pred, y_size, cudaMemcpyDeviceToHost);

	cudaFree(d_pred);
	cudaFree(d_X);
	cudaFree(d_y);
	cudaFree(d_layer_1_delta);
	cudaFree(d_pred_delta);
	cudaFree(d_W0);
	cudaFree(d_W1);
	cudaFree(d_buffer);

	free(h_layer_1_delta);
	free(h_pred_delta);
	free(h_W0);
	free(h_W1);
	free(h_buffer);

	for (int i = 0; i < TRAINING_SIZE; i++){
		printf("Prediction[%i] : %f True Value[%i] : %f Error[%i] : %f\n", i, h_pred[i], i, h_y[i], i, h_pred[i] - h_y[i]);
	}

	free(h_pred);
}

## Why Learn CUDA?
Have a look https://github.com/pytorch/pytorch/tree/5bc44fb6ea65c711c818ea82dc66afee9ad48f78/aten/src/ATen/native/cuda 

## Install CUDA in Colab

In [0]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt-get install cuda gcc-5 g++-5 -y -qq;
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;

In [0]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin


%%cu
#include <iostream>
int main() {
    std::cout << "Hello world\n";
    return 0;
}

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-162d861j
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-162d861j
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=801bf6cbe18412ebc8691bcd3172a7fb9c1d6c1b30193705fd2c8c7e1e2f2427
  Stored in directory: /tmp/pip-ephem-wheel-cache-cdlovexn/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


## Test CUDA Functionality

In [0]:
%%cu
#include <iostream>
int main() {

  std::cout << "Hello World its Siraj";
  return 0;

}

Hello World its Siraj


## Let's write an 'add 2 elements' script in C! 

In [0]:
%%cu
#include <iostream>
#include <math.h>

// function to add the elements of two arrays
__global__ void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
      y[i] = x[i] + y[i];
}

int main(void)
{
 
  int N = 1<<20; // 1M elements

 float *x, *y;


 cudaMallocManaged(&x, N*sizeof(float))
 cudaMallocManaged(&y, N*sizeof(float))


  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }
 

// Run kernel on 1M elements on the CPU
  add(N, x, y);
 


 cudaDeviceSynchronize()

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;
 

  // Free memory
  delete [] x;
  delete [] y;
 

  return 0;
}

Max error: 0



In [0]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true 1.cu -o 1 -lcudadevrt

In [0]:
!nvprof ./1

Max error: 0
======== Warning: No profile data collected.


In [0]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true 2.cu -o 2 -lcudadevrt

gcc: error: 2.cu: No such file or directory
gcc: warning: ‘-x c++’ after last input file has no effect
gcc: fatal error: no input files
compilation terminated.


In [0]:
!nvprof ./2

==240== NVPROF is profiling process 240, command: ./2
Max error: 0
==240== Profiling application: ./2
==240== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  126.13ms         1  126.13ms  126.13ms  126.13ms  add(int, float*, float*)
      API calls:   72.66%  338.61ms         2  169.31ms  45.512us  338.57ms  cudaMallocManaged
                   27.07%  126.15ms         1  126.15ms  126.15ms  126.15ms  cudaDeviceSynchronize
                    0.14%  646.00us         2  323.00us  299.14us  346.86us  cudaFree
                    0.08%  362.23us         1  362.23us  362.23us  362.23us  cuDeviceTotalMem
                    0.03%  157.58us        96  1.6410us     141ns  69.728us  cuDeviceGetAttribute
                    0.01%  41.781us         1  41.781us  41.781us  41.781us  cudaLaunchKernel
                    0.01%  24.340us         1  24.340us  24.340us  24.340us  cuDeviceGetName
                    0.00%  2.

In [0]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true 3.cu -o 3 -lcudadevrt

In [0]:
!nvprof ./3

==294== NVPROF is profiling process 294, command: ./3
Max error: 0
==294== Profiling application: ./3
==294== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  3.9588ms         1  3.9588ms  3.9588ms  3.9588ms  add(int, float*, float*)
      API calls:   98.01%  256.85ms         2  128.42ms  28.643us  256.82ms  cudaMallocManaged
                    1.51%  3.9678ms         1  3.9678ms  3.9678ms  3.9678ms  cudaDeviceSynchronize
                    0.26%  670.17us         2  335.08us  312.04us  358.13us  cudaFree
                    0.14%  368.57us         1  368.57us  368.57us  368.57us  cuDeviceTotalMem
                    0.05%  134.93us        96  1.4050us     126ns  56.333us  cuDeviceGetAttribute
                    0.01%  39.165us         1  39.165us  39.165us  39.165us  cudaLaunchKernel
                    0.01%  23.109us         1  23.109us  23.109us  23.109us  cuDeviceGetName
                    0.00%  3.

In [0]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true 4.cu -o 4 -lcudadevrt

In [0]:
!nvprof ./4

==351== NVPROF is profiling process 351, command: ./4
Max error: 0
==351== Profiling application: ./4
==351== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  3.2025ms         1  3.2025ms  3.2025ms  3.2025ms  add(int, float*, float*)
      API calls:   98.41%  274.11ms         2  137.06ms  35.345us  274.08ms  cudaMallocManaged
                    1.15%  3.2131ms         1  3.2131ms  3.2131ms  3.2131ms  cudaDeviceSynchronize
                    0.21%  575.39us         2  287.70us  258.15us  317.25us  cudaFree
                    0.14%  400.01us         1  400.01us  400.01us  400.01us  cuDeviceTotalMem
                    0.06%  155.55us        96  1.6200us     152ns  64.004us  cuDeviceGetAttribute
                    0.02%  42.642us         1  42.642us  42.642us  42.642us  cudaLaunchKernel
                    0.01%  23.546us         1  23.546us  23.546us  23.546us  cuDeviceGetName
                    0.00%  2.